# Scaling Relations for IrOx systems

***

# Import Modules

In [ ]:
%%capture
%load_ext autoreload
%autoreload 2

In [ ]:
import sys
import os

sys.path.insert(
    0,
    os.path.join(
        os.environ["PROJ_irox"],
        "workflow"))

sys.path.insert(
    0,
    os.path.join(
        os.environ["PROJ_irox"],
        "data"))

from an_data_processing import load_df

###########################################################

# Python Modules
import numpy as np
import pandas as pd

import plotly.plotly as py
import plotly.graph_objs as go

# import colorlover as cl
from IPython.display import HTML
from IPython.display import display

# My Modules
from orr_reaction.orr_fed_plot import ORR_Free_E_Plot
from orr_reaction.orr_fed_plot import Scaling_Relations_Plot

# Project Data
from proj_data_irox import (
    surface_energies,
    smart_format_dict_volcano,

#     color_palettes,
    system_color_map,

    max_surf_e,
    min_surf_e,
    proj_dir_name,
    smart_format_dict,
    data_dir,
    
    groupby_props,
    
    )

# Script Inputs

In [ ]:
save_plot = True

prop_name_list = [
    'bulk_system',
#     'coverage',
    'coverage_type',
    'facet',
    ]

# Read and Process Data Frame

In [ ]:
df_pourbaix, df_ads, df_surf = load_df(
    from_file=False,
    root_dir=data_dir,
    data_dir=data_dir,
    file_name="df_master.pickle",
    process_df=True,
    )

df_m = df_ads

df_m.loc[df_m["coverage_type"] == "O-4_OH-0", "coverage_type"] = "o_covered"
df_m.loc[df_m["coverage_type"] == "O-2_OH-0", "coverage_type"] = "o_covered_2"
df_m.loc[df_m["coverage_type"] == "O-2_OH-2", "coverage_type"] = "h_covered"

## Remove *OOH rows that aren't pointing "up" or "sideways" is also acceptable
I think for now it'll be fine to only use *OOH up structures

In [ ]:
grouped = df_m.groupby(groupby_props)

ignore_indices = []
for i_ind, (name, group) in enumerate(grouped):
    df_i = group
    
    df_ooh_i = df_i[df_i["adsorbate"] == "ooh"]
    
    if len(df_ooh_i) > 1:
        if "up" in df_ooh_i["ooh_direction"].tolist():
            ignore_indices_i = list(df_ooh_i[df_ooh_i["ooh_direction"] != "up"].index.values)
            ignore_indices += ignore_indices_i
        elif "sideways" in df_ooh_i["ooh_direction"].tolist():
            ignore_indices_i = list(df_ooh_i[df_ooh_i["ooh_direction"] != "sideways"].index.values)
            ignore_indices += ignore_indices_i
        else:
            tmp = 42

df_m = df_m.drop(labels=ignore_indices)

In [ ]:

# grouped = df_tmp.groupby(groupby_props)
# ignore_indices = []
# for i_ind, (name, group) in enumerate(grouped):
#     df_i = group
    
#     df_ooh_i = df_i[df_i["adsorbate"] == "ooh"]
    
#     print(len(df_ooh_i))
    
#     if len(df_ooh_i) > 1:
#         display(df_ooh_i)

# if len(df_ooh_i) > 1:
#     print("yes!")

#     if "up" in df_ooh_i["ooh_direction"].tolist():
#         print("yes 2!")

#         ignore_indices_i = list(df_ooh_i[df_ooh_i["ooh_direction"] != "up"].index.values)
#         ignore_indices += ignore_indices_i

# list(df_ooh_i[df_ooh_i["ooh_direction"] != "up"].index.values)

# df_m["ooh_direction"]
# df_m[
#     (df_m["adsorbate"] == "ooh") &
# #     (df_m["facet"] == "100") &
# #     (df_m["coverage_type"] == "o_covered") &
#     [True for i in range(len(df_m))]
#     ]

# df_m[
#     (df_m["bulk_system"] == "IrO2") &
#     (df_m["facet"] == "100") &
#     (df_m["coverage_type"] == "o_covered") &
#     [True for i in range(len(df_m))]
#     ]

# ** Fitting only O-covered data

In [ ]:
df_o = df_m[df_m["coverage_type"] == "o_covered"]
df_m_tmp = df_o

ORR_PLT = ORR_Free_E_Plot(
    free_energy_df=None,
    state_title="adsorbate",
    free_e_title="ads_e",
    smart_format=smart_format_dict,
    bias=0.,
    color_list=None,   
    show_H_e_pairs_annotations=True,
    show_legend=True,
    rxn_type="OER",
    )

grouped = df_m_tmp.groupby(groupby_props)


for i_ind, (name, group) in enumerate(grouped):
    df_i = group

    # Choosing the most stable *OOH species
    # ###################################################
    species_j = "ooh"
    df_wo_species = df_i[df_i["adsorbate"] != species_j]
    df_ij = df_i[df_i["adsorbate"] == species_j]
    df_final = df_wo_species.append(
        df_ij.loc[df_ij["ads_e"].idxmin()]
        )
    df_final

    df_i = df_final
    # ###################################################

    sys_i = df_i.iloc[0]["bulk_system"] + "_" + df_i.iloc[0]["facet"]
    color_i = system_color_map[sys_i]
    
    if not any([np.isnan(i) for i in df_i.elec_energy.tolist()]):        
        ORR_PLT.add_series(
            df_i,
            plot_mode="all",
#             opt_name=df_i["name_i"].tolist()[0],
            overpotential_type="OER",
            property_key_list=prop_name_list,
            add_overpot=False,
#             color=color_i,
            )
        
SC_PLT = Scaling_Relations_Plot(
    ORR_PLT,
    plot_range={
        "y": [0., 5.],
        "x": [0., 1.8]})

slope_intercept_dict_ooh_o_covered = SC_PLT.fit_scaling_lines("ooh")
slope_intercept_dict_o_o_covered = SC_PLT.fit_scaling_lines("o")

# ** Fitting only H-covered data

In [ ]:
df_h = df_m[df_m["coverage_type"] == "h_covered"]
df_m_tmp = df_h

ORR_PLT = ORR_Free_E_Plot(
    free_energy_df=None,
    state_title="adsorbate",
    free_e_title="ads_e",
    smart_format=smart_format_dict,
    bias=0.,
    color_list=None,   
    show_H_e_pairs_annotations=True,
    show_legend=True,
    rxn_type="OER",
    )

grouped = df_m_tmp.groupby(groupby_props)


for i_ind, (name, group) in enumerate(grouped):
    df_i = group
#     df_dict["_".join(list(name))] = df_i

    # Choosing the most stable *OOH species
    # ###################################################
    species_j = "ooh"
    df_wo_species = df_i[df_i["adsorbate"] != species_j]
    df_ij = df_i[df_i["adsorbate"] == species_j]
    df_final = df_wo_species.append(
        df_ij.loc[df_ij["ads_e"].idxmin()]
        )
    df_final

    df_i = df_final
    # ###################################################

    sys_i = df_i.iloc[0]["bulk_system"] + "_" + df_i.iloc[0]["facet"]
    color_i = system_color_map[sys_i]
    
    if not any([np.isnan(i) for i in df_i.elec_energy.tolist()]):        
        ORR_PLT.add_series(
            df_i,
            plot_mode="all",
            overpotential_type="OER",
            property_key_list=prop_name_list,
            add_overpot=False,
#             color=color_i,
            )
        
SC_PLT = Scaling_Relations_Plot(
    ORR_PLT,
    plot_range={
        "y": [0., 5.],
        "x": [0., 1.8]})

slope_intercept_dict_ooh_h_covered = SC_PLT.fit_scaling_lines("ooh")
slope_intercept_dict_o_h_covered = SC_PLT.fit_scaling_lines("o")

# ORR_Free_E_Plot Instance

In [ ]:
ORR_PLT = ORR_Free_E_Plot(
    free_energy_df=None,
    state_title="adsorbate",
    free_e_title="ads_e",
    smart_format=smart_format_dict,
    bias=0.,
    color_list=None,   
    show_H_e_pairs_annotations=True,
    show_legend=True,
    rxn_type="OER",
    )

# Processing Data

In [ ]:
grouped = df_m.groupby(groupby_props)

df_dict = {}
for i_ind, (name, group) in enumerate(grouped):
    df_i = group

    # Choosing the most stable *OOH species
    # ###################################################
    species_j = "ooh"
    df_wo_species = df_i[df_i["adsorbate"] != species_j]
    df_ij = df_i[df_i["adsorbate"] == species_j]
    df_final = df_wo_species.append(
        df_ij.loc[df_ij["ads_e"].idxmin()]
        )
    df_final

    df_i = df_final
    # ###################################################

    df_dict["_".join(list(name))] = df_i
      
    sys_i = df_i.iloc[0]["bulk_system"] + "_" + df_i.iloc[0]["facet"]
    color_i = system_color_map[sys_i]
            
    if not any([np.isnan(i) for i in df_i.elec_energy.tolist()]):        
        ORR_PLT.add_series(
            df_i,
            plot_mode="all",
#             opt_name=df_i["name_i"].tolist()[0],
            overpotential_type="OER",
            property_key_list=prop_name_list,
            add_overpot=False,
#             color=color_i,
            )

# Scaling_Relations_Plot Instance

In [ ]:
SC_PLT = Scaling_Relations_Plot(
    ORR_PLT,
    plot_range={
        "y": [0., 5.],
        "x": [0., 1.8],
        },

    )

# exclude_dict = {
#     'bulk_system': 'IrO2',
#     'coverage_type': 'o_covered',
# #     'dopant': 'Ni',
# #     'facet': '110',
# #     'site': 'ir_site',
#     'site': 'ir_site',
#     }

slope_intercept_dict = SC_PLT.fit_scaling_lines(
    "ooh",
#     exclude_dict=exclude_dict,
    )

SC_PLT.add_line(slope_intercept_dict,
    name="*OOH vs *OH Scaling",
    color="red",
    width=1,
    dash="solid",
    )

slope_intercept_dict = SC_PLT.fit_scaling_lines(
    "o",
#     exclude_dict=exclude_dict,
    )

SC_PLT.add_line(slope_intercept_dict,
    name="*O vs *OH Scaling",
    color="red",
    width=1,
    dash="solid",
    )

In [ ]:
# slope_intercept_dict_ooh_o_covered
SC_PLT.add_line(slope_intercept_dict_ooh_o_covered,
    name="ooh_vs_oh | o_covered",
    color="orange",
    width=1,
    dash="solid",
    )

SC_PLT.add_line(slope_intercept_dict_o_o_covered,
    name="o_vs_oh | o_covered",
    color="orange",
    width=1,
    dash="solid",
    )

# slope_intercept_dict_ooh_o_covered
SC_PLT.add_line(slope_intercept_dict_ooh_h_covered,
    name="ooh_vs_oh | h_covered",
    color="yellow",
    width=1,
    dash="dash",
    )

SC_PLT.add_line(slope_intercept_dict_o_h_covered,
    name="o_vs_oh | h_covered",
    color="yellow",
    width=1,
    dash="dash",
    )

In [ ]:
SC_PLT.add_ideal_lines()

SC_PLT.create_scaling_relations_plot(
    smart_format_dict=smart_format_dict,
    )

data_m = SC_PLT.data_points["ooh_vs_oh"] + \
    SC_PLT.data_points["o_vs_oh"] + \
    SC_PLT.data_points["oh_vs_oh"] + \
    SC_PLT.data_lines

# Plot

In [ ]:
tick_lab_size = 8 * (4. / 3.)
axes_lab_size = 9 * (4. / 3.)

layout_override = {
#     "title": "KLSFJKSDJKFJSKDF",

    "width": 1.5 * 7.964 * 37.795275591,
    "height": 1.5 * 5.6002 * 37.795275591,

    "yaxis": {
        "titlefont": {
            "size": axes_lab_size,
            },
        "tickfont": {
            "size": tick_lab_size,
            },
        },


    "xaxis": {
        "titlefont": {
            "size": axes_lab_size,
            },
        "tickfont": {
            "size": tick_lab_size,
            },
        },
    
#     "margin": go.layout.Margin(
#         b=50.,
#         l=50.,
#         r=50.,
#         t=50.,        
#         ), 

    }

In [ ]:
layout = SC_PLT.__create_layout__(
#     x_ax_spec="oh",
    title=None,
    showlegend=False,
    layout_dict=layout_override,
    )
layout["annotations"] = SC_PLT.annotations_list

if save_plot:
    save_dir = proj_dir_name
else:
    save_dir = "__temp__"

fig = dict(data=data_m, layout=layout)
# py.iplot(fig, filename=os.path.join(save_dir, "pl_scaling_relations"))

py.iplot(
    fig,
    filename=os.path.join(
        save_dir,
        "scaling_relations",
        "pl_scaling_relations",
        )
    )

In [ ]:
layout = SC_PLT.__create_layout__(
#     x_ax_spec="oh",
    title="Scaling Relations",
    showlegend=True,
    )

layout["margin"] = go.layout.Margin(
    b=50.,
    l=50.,
    r=50.,
    t=50.,
#     pad=20.,
    )

layout["width"] = 4.5 * 7.964 * 37.795275591
layout["height"] = 5.5 * 5.6002 * 37.795275591

layout["annotations"] = SC_PLT.annotations_list

In [ ]:
if save_plot:
    save_dir = proj_dir_name
else:
    save_dir = "__temp__"

fig = dict(data=data_m, layout=layout)
py.iplot(
    fig,
    filename=os.path.join(
        save_dir,
        "scaling_relations",
        "pl_scaling_relations_large",
        )
    )

# Fitting to O-covered and H-covered Separately

In [ ]:
df_h = df_m[df_m["coverage_type"] == "h_covered"]

In [ ]:
df_o = df_m[df_m["coverage_type"] == "o_covered"]
df_m_tmp = df_o

ORR_PLT = ORR_Free_E_Plot(
    free_energy_df=None,
    state_title="adsorbate",
    free_e_title="ads_e",  
    smart_format=smart_format_dict,
    bias=0.,
    color_list=None,   
    show_H_e_pairs_annotations=True,
    show_legend=True,
    rxn_type="OER",
    )

grouped = df_m_tmp.groupby(groupby_props)


for i_ind, (name, group) in enumerate(grouped):
    df_i = group
#     df_dict["_".join(list(name))] = df_i

    sys_i = df_i.iloc[0]["bulk_system"] + "_" + df_i.iloc[0]["facet"]
    color_i = system_color_map[sys_i]
    
    if not any([np.isnan(i) for i in df_i.elec_energy.tolist()]):        
        ORR_PLT.add_series(
            df_i,
            plot_mode="all",
#             opt_name=df_i["name_i"].tolist()[0],
            overpotential_type="OER",
            property_key_list=prop_name_list,
            add_overpot=False,
#             color=color_i,
            )
        
SC_PLT = Scaling_Relations_Plot(
    ORR_PLT,
    plot_range={
        "y": [0., 5.],
        "x": [0., 1.8]})

slope_intercept_dict_ooh_o_covered = SC_PLT.fit_scaling_lines("ooh")
slope_intercept_dict_o_o_covered = SC_PLT.fit_scaling_lines("o")

In [ ]:
# SC_PLT.add_line(slope_intercept_dict,
#     name="ooh_vs_oh | o_covered",
#     color="orange",
#     width=1,
#     dash="solid",
#     )

# SC_PLT.add_line(slope_intercept_dict,
#     name="o_vs_oh | o_covered",
#     color="red",
#     width=1,
#     dash="solid",
#     )

# SC_PLT.add_ideal_lines()

# SC_PLT.create_scaling_relations_plot(
#     smart_format_dict=smart_format_dict,
#     )

# data_m = SC_PLT.data_points["ooh_vs_oh"] + \
#     SC_PLT.data_points["o_vs_oh"] + \
#     SC_PLT.data_points["oh_vs_oh"] + \
#     SC_PLT.data_lines

In [ ]:
tick_lab_size = 8 * (4. / 3.)
axes_lab_size = 9 * (4. / 3.)

layout_override = {
#     "title": "KLSFJKSDJKFJSKDF",

    "width": 1.5 * 7.964 * 37.795275591,
    "height": 1.5 * 5.6002 * 37.795275591,

    "yaxis": {
        "titlefont": {
            "size": axes_lab_size,
            },
        "tickfont": {
            "size": tick_lab_size,
            },
        },


    "xaxis": {
        "titlefont": {
            "size": axes_lab_size,
            },
        "tickfont": {
            "size": tick_lab_size,
            },
        },

#     "margin": go.layout.Margin(
#         b=50.,
#         l=50.,
#         r=50.,
#         t=50.,        
#         ), 

    }

layout = SC_PLT.__create_layout__(
#     x_ax_spec="oh",
    title=None,
    showlegend=False,
    layout_dict=layout_override,
    )
layout["annotations"] = SC_PLT.annotations_list

if save_plot:
    save_dir = proj_dir_name
else:
    save_dir = "__temp__"

fig = dict(data=data_m, layout=layout)
py.iplot(
    fig,
    filename=os.path.join(
        save_dir,
        "scaling_relations",
        "pl_scaling_relations_o_covered",
        )
    )

In [ ]:
save_dir

In [ ]:
# df_m[
#     (df_m["bulk_system"] == "IrO3_rutile-like") &
#     (df_m["facet"] == "100") &
#     (df_m["coverage_type"] == "o_covered_2") &
#     [True for i in range(len(df_m))]
#     ]